In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
mean_1 = np.ones(10)*0.5
mean_2 = np.zeros(10) 
num_samples = 150
num_test=100

COV = np.diag(np.ones(10))
print(sigma)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:

sample_1 = np.random.multivariate_normal(mean=mean_1,cov=COV,size=num_samples)
sample_2=np.random.multivariate_normal(mean=mean_2,cov=COV,size=num_samples)

test_1 = np.random.multivariate_normal(mean=mean_1,cov=COV,size=num_test)
test_2 = np.random.multivariate_normal(mean=mean_2,cov=COV,size=num_test)

# Now that we have our two datasets, let's "mix them up" and assign the relavant classes
# This is our x vector 
training_data = np.concatenate((sample_1,sample_2))

# Labels will be 0 for dataset 1, 1 for dataset 2
# This is basically our y vector
training_data_labels = np.concatenate((np.zeros(num_samples), np.ones(num_samples)))


test_data = np.concatenate((test_1,test_2))

test_labels = np.concatenate((np.zeros(num_test), np.ones(num_test)))

# Produce the sigmoid function

def sigma(a):
    return 1/(1+np.exp(a))


# vec_mu is a function of vec_w and our datapoints 

def cross_entropy(p,q):
    return -(p*np.log(q)+(1-p)*np.log(1-q))

def NLL(vec_w, bias, vec_y, mtx_X, l):
    to_ret = 0
    N, C = np.array(mtx_X).shape

    # Modify to absorb bias 
    mtx_X = [np.concatenate((x,[1])) for x in mtx_X]
    for n in range(N):
        mu_n = sigma(vec_w @ mtx_X[n]+bias)
        to_ret+=cross_entropy(vec_y[n],mu_n)
    return  1/N*to_ret+l*np.dot(vec_w,vec_w)


def grad_NLL(vec_w, vec_y, mtx_X, l):
    N, C = np.array(mtx_X).shape
    # Modify to absorb bias 
    mtx_X= [np.concatenate((x, [1])) for x in mtx_X]

    vec_mu = [sigma(x) for x in mtx_X @ vec_w]
    return (np.transpose(np.ones(N)) @ np.diag(np.array(vec_mu)-np.array(vec_y)) @ mtx_X)/N+2*l*vec_w


# With our grad created, we can now perform grad descent

# We know L(theta) to be our negative log likelihood fn we're trying to mimimize, so let's create a basic SGD algorithm

def SGD(vec_y, mtx_X, num_iterations, step_size,l):
    N, C = np.array(mtx_X).shape
    # Create a theta with weights and biases incorporated
    theta = np.concatenate((np.ones(C),[1]))
    random_sample_size = 10
    for k in range(num_iterations):
        # Get a random sample of data points 
        random_samples = [np.random.randint(0,N) for i in range(random_sample_size)]
        # Add a vector onto theta 
        vec_y_subset = [vec_y[random_samples[i]] for i in range(random_sample_size)]
        mtx_X_subset = [mtx_X[random_samples[i]] for i in range(random_sample_size)]

        theta = theta-step_size*grad_NLL(theta, vec_y_subset,mtx_X_subset,l)
    return theta

def model(sample_vector, weights):
    # Allow for bias
    sample_vector = np.concatenate((sample_vector,[1]))
    return sigma(np.dot(weights,sample_vector))

def provide_guess(sample_vector, weights):
    probability = model(sample_vector=sample_vector, weights=weights)
    return 1 if np.random.uniform(0,1)>probability else 0

def calculate_accuracy(data, data_labels, weights):
    confusion_matrix = [[0,0],[0,0]]
    N=len(data)
    guesses = np.zeros(N)
    for i in range(N): 
        guesses[i] = provide_guess(sample_vector=data[i], weights=weights)
        if (guesses[i]==data_labels[i]):
            if guesses[i]==0:
                confusion_matrix[0][0]+=1
            if guesses[i]==1:
                confusion_matrix[1][1]+=1
        else:
            if guesses[i]==1:
                confusion_matrix[0][1]+=1
            if guesses[i]==0:
                confusion_matrix[1][0]+=1
    return confusion_matrix, (confusion_matrix[0][0]+confusion_matrix[1][1])/N

# Create a grid of points (here we'll make a 2D grid)
x = np.linspace(-10, 10, 1000)
y = np.linspace(-10, 10, 1000)
X, Y = np.meshgrid(x, y)

# Create a 2D array to hold the results of the function
Z = np.zeros_like(X)

# Now, calculate ensemble of models (in this case 100 diff models)

l_values = np.linspace(0,10,1000)

ensemble_models = [SGD(vec_y=training_data_labels,mtx_X=training_data, num_iterations=1000,step_size=0.01, l=l_index) for l_index in l_values]

accuracy = [calculate_accuracy(data=test_data,data_labels=test_labels,weights=model) for model in ensemble_models]


plt.figure()
plt.plot(np.log(l_values),[x[1] for x in accuracy],'o', markersize=3)
plt.title("Accuracy of model vs lambda")
plt.xlabel("log(lambda)")
plt.ylabel("Accuracy")

weights = SGD(vec_y=training_data_labels,mtx_X=training_data,num_iterations=1000,step_size=2,l=0.2)

lambda_max = np.argmax([x[1] for x in accuracy])
print(l_values[lambda_max])

# Loop through the grid and apply model to each vector (x, y)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = model([X[i, j], Y[i, j]], ensemble_models[lambda_max])  # Pass each grid point as a vector

plt.figure()
plt.title("Scatter plot and decision boundary using MAP Logistic Regression")
x_1,y_1=zip(*sample_1)
x_2,y_2=zip(*sample_2)

plt.plot(x_1,y_1,'o', markersize=3)
plt.plot(x_2,y_2,'x',markersize=3)

plt.imshow(Z,extent=[-10, 10, -10, 10], origin='lower', cmap='coolwarm', alpha=0.5)
plt.show()
